In [1]:
# !pip install --user unidecode
import unidecode
import random
import operator
import math
accented_string = u'Málaga żródło Dóżź←ęq"'
# accented_string is of type 'unicode'

unaccented_string = unidecode.unidecode(accented_string)
unaccented_string

'Malaga zrodlo Dozz-eq"'

In [2]:
def remove_nonalpha(s, fun=str.isalnum):
    return ''.join(filter(fun, s))

def remove_nonalpha_from_sent(sent):
    return remove_nonalpha(sent, fun=lambda x: x ==' ' or str.isalnum(x))

def normalize(s):
    return remove_nonalpha(unidecode.unidecode(s).lower())

def normalize_sentence(s):
    return remove_nonalpha_from_sent(unidecode.unidecode(s).lower())


    
def loader(fname, cut = 0, top = float('inf')):
    data = dict()
    decodeData = {}
    with open(fname, 'r') as f:
        i=0
        for x in f:
            if not i%1000000:
                print(i)
            i+=1
            if i > top:
                break
            num, key, key2 = x.split()
            
            if int(num) > cut:
                d_key1 = normalize(key)
                d_key2 = normalize(key2)
                
                if d_key1 not in decodeData:
                    decodeData[d_key1] = dict()
                    
                if d_key2 not in decodeData:
                    decodeData[d_key2] = dict()
                
                if remove_nonalpha(key) not in decodeData[d_key1]:
                    decodeData[d_key1][remove_nonalpha(key)] = 1
                else:
                    decodeData[d_key1][remove_nonalpha(key)] += 1
                
                if remove_nonalpha(key2) not in decodeData[d_key2]:
                    decodeData[d_key2][remove_nonalpha(key2)] = 1
                else:
                    decodeData[d_key2][remove_nonalpha(key2)] += 1
                
                if key not in data:
                    data[key] = dict()
                data[key][key2] = int(num)
    return data, decodeData

def get_corpus(fname):
    return open(fname).read().split('\n')

def random_sentence(corpus):
    return random.choice(corpus)

# def 

In [3]:
# d1, d2 = loader('/pio/data/data/poleval_polish_corpora/poleval_2grams.txt')
# corpus = get_corpus('/pio/data/data/poleval_polish_corpora/polish_corpora.txt')

bigramsInfo, alteranateForms = loader('../1/poleval_2grams.txt', cut=15)
corpus = get_corpus('sample-corpus')


0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000
13000000
14000000
15000000
16000000
17000000
18000000
19000000
20000000
21000000
22000000
23000000
24000000
25000000
26000000
27000000
28000000
29000000
30000000
31000000
32000000
33000000
34000000
35000000
36000000
37000000
38000000
39000000
40000000
41000000
42000000
43000000
44000000
45000000
46000000
47000000
48000000
49000000
50000000
51000000
52000000
53000000
54000000
55000000
56000000
57000000
58000000
59000000


In [24]:
def create_alts(sent):
    alts = []
    for i in sent:
        if i in alteranateForms:
            alts.append({x:[alteranateForms[i][x]] for x in alteranateForms[i]})
        else:
            alts.append({i:[1]})
    return alts

def max_previous_bi(prevs, word):
    maxx = 0
    for w in prevs:
        if w in bigramsInfo:
            if word in bigramsInfo[w]:
                if int(bigramsInfo[w][word]) > maxx:
                    maxx = int(bigramsInfo[w][word])
    return maxx

def max_next_bi(nexts, word):
    maxx = 0
    if word in bigramsInfo:
        for w in nexts:
            if w in bigramsInfo[word]:
                if int(bigramsInfo[word][w]) > maxx:
                    maxx = int(bigramsInfo[word][w])
    return maxx

def add_bigrams_info(alts):
    altsExt = [{'<BOS>':[1]}] + alts  + [{'<EOS>':[1]}]
    for num, (w0,w1,w2) in enumerate(zip(altsExt, altsExt[1:], altsExt[2:])):
        for word in w1:
            maxPrev = max_previous_bi(w0,word)
            alts[num][word].append(maxPrev)
            maxNext = max_next_bi(w2,word)
            alts[num][word].append(maxNext)
    
#     for i in alts:
#         print(i)
        
            
    return alts

def normalize_answer(alts, target=1.0):
    raws = []
    if len(alts)>0:
        for i in range(len(list(alts[0].values())[0])):
            raw = sum([x[i] for y in alts for x in y.values()])
            raw = raw or 1
            raws.append(raw)
            
        alts = [
            {
                k: [
                    wAlts[k][k2] / raws[k2]  
                    for k2 in range(len(wAlts[k])) 
                ] for k in wAlts
            } for wAlts in alts
        ]    
    return alts

def count_weights(tab, a=1, b=10, c=10):
    return a * tab[0] + b * tab[1] + c * tab[2]

def generate_answer(alts, a=1, b=10, c=10):
    answ = []
    for words in alts:
        answ.append(
            max(
                {
                    k:count_weights(words[k], a, b, c) 
                    for k in words
                }.items(), 
                key=operator.itemgetter(1))[0]
        )
    return answ

def get_score(sent, answer):
    if len(sent) > 0:
#         for i in range(len(sent)):
#             if sent[i] != answer[i]:
#                 print('\n\n', sent[i], answer[i])
        return len([i for i in range(len(sent)) if sent[i] == answer[i]])/len(sent)
    return 1


In [38]:
sent = random_sentence(corpus)

sent_to_test = remove_nonalpha_from_sent(sent).split()
test = normalize_sentence(sent)
test = test.split()
print(sent_to_test)
print()
alts = create_alts(test)
maxx = 0
bests = []
for i in range(1, 20):
    print(i)
    for j in range(1, 20):
        a = normalize_answer(add_bigrams_info(alts))
        for k in range(1, 20):
            ans = generate_answer(a, i,j,k)
            ans[0] = ans[0].capitalize()
            # print(ans)
            score_polish = get_score([i.lower() for i in sent_to_test], [i.lower() for i in ans])
            score_full = get_score(sent_to_test, ans)
            score = math.sqrt(score_polish * score_full)
#             print(score)
            if score > maxx:
                maxx = score
                bests = [i,j,k]
                print(maxx)
            if maxx == 1:
                break
        if maxx == 1:
            break
    if maxx == 1:
        break
print(maxx, bests)

['Mimo', 'iż', 'takie', 'miejsca', 'tracą', 'powoli', 'dawną', 'popularność', 'wielu', 'współczesnym', 'działaczom', 'polskiej', 'turystyki', 'marzą', 'się', 'podobne', 'widoki', 'w', 'Polsce', 'i', 'chcą', 'przekształcić', 'często', 'niepowtarzalny', 'krajobraz', 'w', 'betonowe', 'ośrodki', 'rozrywki', 'sportu', 'i', 'wypoczynku', 'do', 'których', 'za', 'kilkadziesiąt', 'lat', 'nikt', 'nie', 'będzie', 'chciał', 'przyjeżdżać']

1
0.9642122253007898
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.9642122253007898 [1, 1, 1]
